In [33]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import time
from scipy import stats 
from sklearn.datasets.samples_generator import make_regression
from sklearn.model_selection import train_test_split 

%matplotlib inline

In [34]:
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

In [35]:
bags = pd.read_csv ('Data_miniproject.csv');
bags = bags[bags.Cost != 0]
# y = bags.pop('Cost');
# X = bags.pop('Height')
X = bags.iloc[:,1:6]
X = X.values
y = bags.iloc[:,0]
y=y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [36]:
def cal_cost(theta,X,y):
    m = len(y)
    predictions = X.dot(theta)
    cost = (1/2*m) * np.sum(np.square(predictions-y))
    return cost

In [61]:
@timeit
def gradient_descent(X, y, theta, learning_rate=0.01, iterations=100):
    m = len(y)
    n = X.shape[1]
    cost_history = np.zeros(iterations)
    theta_history = np.zeros((iterations, n))
    for it in range(iterations):
        prediction = np.dot(X, theta)
        theta = theta - (1 / m) * learning_rate * (X.T.dot((prediction - y)))
        theta_history[it, :] = theta.T
        cost_history[it] = cal_cost(theta, X, y)
    return theta, cost_history, theta_history

In [63]:
lr = 0.05
n_iter = 1000
m,n=np.shape(X)
y=y.reshape(m,1)
x=np.c_[np.ones((m,1)),X]
theta=np.zeros((n+1,1))
theta
# # theta = np.random.randn(n+1,1)
X_b = np.c_[np.ones((len(X), 1)), X]
X_b
# # np.concatenate((np.ones((X_train.shape[0],1)), X_train),axis = 1)
theta, cost_history, theta_history = gradient_descent(X_b, y, theta, lr, n_iter)
print("Theta0: {:0.3f},\nTheta1:{:0.3f}".format(theta[0][0], theta[1][0]))
print("Final cost/MSE:  {:0.3f}".format(cost_history[-1]))

'gradient_descent'  55.24 ms
Theta0: nan,
Theta1:nan
Final cost/MSE:  nan


/home/psk/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: overflow encountered in square
  after removing the cwd from sys.path.
/home/psk/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in subtract
  if __name__ == '__main__':


In [41]:
@timeit
def minibatch_gradient_descent(X,y,theta,learning_rate=0.01, iterations=10, batch_size =20):
    m = len(y)
    cost_history = np.zeros(iterations)
    n_batches = int(m/batch_size)
    for it in range(iterations):
        cost =0.0
        indices = np.random.permutation(m)
        X = X[indices]
        y = y[indices]
        for i in range(0,m,batch_size):
            X_i = X[i:i+batch_size]
            y_i = y[i:i+batch_size]
            X_i = np.c_[np.ones(len(X_i)),X_i]
            prediction = np.dot(X_i,theta)
            theta = theta -(1/m)*learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta,X_i,y_i)
        cost_history[it]  = cost
    return theta, cost_history

In [46]:
lr = 0.05
n_iter = 1000
theta = np.random.randn(20, 6)
theta
# theta,cost_history = minibatch_gradient_descent(X,y,theta,lr,n_iter)
# print('Theta0:          {:0.3f},\nTheta1:          {:0.3f}'.format(theta[0][0],theta[1][0]))
# print('Final cost/MSE:  {:0.3f}'.format(cost_history[-1]))


array([[ 0.95194575,  0.10761014, -0.7402122 , -0.08933266,  0.96175112,
         1.03643556],
       [ 0.87462758,  0.36477309,  0.91366881, -0.60920934, -1.48817501,
        -1.10577411],
       [-0.25225937,  0.19087009,  0.86493494,  0.38999984, -0.02885042,
         1.27919763],
       [-0.55466363, -1.52657066,  0.76803921,  0.72293479,  1.04313737,
        -0.18867175],
       [ 1.53947623, -0.18223603, -1.52513937, -0.23033542, -1.74859774,
         0.76278628],
       [ 0.00703904, -0.77936172,  0.9850332 , -1.24012242,  0.27085325,
         0.70402237],
       [-0.0617313 ,  0.04305232, -0.42133041,  0.42668344,  0.02904304,
         1.05656925],
       [ 1.06731   ,  0.1164101 ,  0.40668145,  1.45578955, -0.08156634,
        -0.31416297],
       [-1.56066022,  0.47688654, -1.15966885, -0.70922387, -0.31628381,
        -0.43114963],
       [ 0.02683375,  0.16061389, -0.42780901,  0.43853202,  1.28615936,
         0.42004406],
       [ 0.49389609, -0.14996768, -1.3685626 , -1.